The following additional libraries are needed to run this
notebook. Note that running on Colab is experimental, please report a Github
issue if you have any problem.

In [ ]:
!pip install d2l==0.17.0

In [ ]:
!pip install tensorflow==2.8.0

# Long Short-Term Memory (LSTM)
:label:`sec_lstm`

The challenge to address long-term information preservation and short-term input
skipping in latent variable models has existed for a long time. One of the
earliest approaches to address this was the
long short-term memory (LSTM) :cite:`Hochreiter.Schmidhuber.1997`. It shares many of the properties of the
GRU.
Interestingly, LSTMs have a slightly more complex
design than GRUs but predates GRUs by almost two decades.



## Gated Memory Cell

Arguably LSTM's design is inspired
by logic gates of a computer.
LSTM introduces a *memory cell* (or *cell* for short)
that has the same shape as the hidden state
(some literatures consider the memory cell
as a special type of the hidden state),
engineered to record additional information.
To control the memory cell
we need a number of gates.
One gate is needed to read out the entries from the
cell.
We will refer to this as the
*output gate*.
A second gate is needed to decide when to read data into the
cell.
We refer to this as the *input gate*.
Last, we need a mechanism to reset
the content of the cell, governed by a *forget gate*.
The motivation for such a
design is the same as that of GRUs,
namely to be able to decide when to remember and
when to ignore inputs in the hidden state via a dedicated mechanism. Let us see
how this works in practice.


### Input Gate, Forget Gate, and Output Gate

Just like in GRUs,
the data feeding into the LSTM gates are
the input at the current time step and
the hidden state of the previous time step,
as illustrated in :numref:`lstm_0`.
They are processed by
three fully-connected layers with a sigmoid activation function to compute the values of
the input, forget. and output gates.
As a result, values of the three gates
are in the range of $(0, 1)$.

![Computing the input gate, the forget gate, and the output gate in an LSTM model.](https://github.com/d2l-ai/d2l-tensorflow-colab-classic/blob/master/img/lstm-0.svg?raw=1)
:label:`lstm_0`

Mathematically,
suppose that there are $h$ hidden units, the batch size is $n$, and the number of inputs is $d$.
Thus, the input is $\mathbf{X}_t \in \mathbb{R}^{n \times d}$ and the hidden state of the previous time step is $\mathbf{H}_{t-1} \in \mathbb{R}^{n \times h}$. Correspondingly, the gates at time step $t$
are defined as follows: the input gate is $\mathbf{I}_t \in \mathbb{R}^{n \times h}$, the forget gate is $\mathbf{F}_t \in \mathbb{R}^{n \times h}$, and the output gate is $\mathbf{O}_t \in \mathbb{R}^{n \times h}$. They are calculated as follows:

$$
\begin{aligned}
\mathbf{I}_t &= \sigma(\mathbf{X}_t \mathbf{W}_{xi} + \mathbf{H}_{t-1} \mathbf{W}_{hi} + \mathbf{b}_i),\\
\mathbf{F}_t &= \sigma(\mathbf{X}_t \mathbf{W}_{xf} + \mathbf{H}_{t-1} \mathbf{W}_{hf} + \mathbf{b}_f),\\
\mathbf{O}_t &= \sigma(\mathbf{X}_t \mathbf{W}_{xo} + \mathbf{H}_{t-1} \mathbf{W}_{ho} + \mathbf{b}_o),
\end{aligned}
$$

where $\mathbf{W}_{xi}, \mathbf{W}_{xf}, \mathbf{W}_{xo} \in \mathbb{R}^{d \times h}$ and $\mathbf{W}_{hi}, \mathbf{W}_{hf}, \mathbf{W}_{ho} \in \mathbb{R}^{h \times h}$ are weight parameters and $\mathbf{b}_i, \mathbf{b}_f, \mathbf{b}_o \in \mathbb{R}^{1 \times h}$ are bias parameters.

### Candidate Memory Cell

Next we design the memory cell. Since we have not specified the action of the various gates yet, we first introduce the *candidate* memory cell $\tilde{\mathbf{C}}_t \in \mathbb{R}^{n \times h}$. Its computation is similar to that of the three gates described above, but using a $\tanh$ function with a value range for $(-1, 1)$ as the activation function. This leads to the following equation at time step $t$:

$$\tilde{\mathbf{C}}_t = \text{tanh}(\mathbf{X}_t \mathbf{W}_{xc} + \mathbf{H}_{t-1} \mathbf{W}_{hc} + \mathbf{b}_c),$$

where $\mathbf{W}_{xc} \in \mathbb{R}^{d \times h}$ and $\mathbf{W}_{hc} \in \mathbb{R}^{h \times h}$ are weight parameters and $\mathbf{b}_c \in \mathbb{R}^{1 \times h}$ is a bias parameter.

A quick illustration of the candidate memory cell is shown in :numref:`lstm_1`.

![Computing the candidate memory cell in an LSTM model.](https://github.com/d2l-ai/d2l-tensorflow-colab-classic/blob/master/img/lstm-1.svg?raw=1)
:label:`lstm_1`

### Memory Cell

In GRUs, we have a mechanism to govern input and forgetting (or skipping).
Similarly,
in LSTMs we have two dedicated gates for such purposes: the input gate $\mathbf{I}_t$ governs how much we take new data into account via $\tilde{\mathbf{C}}_t$ and the forget gate $\mathbf{F}_t$ addresses how much of the old memory cell content $\mathbf{C}_{t-1} \in \mathbb{R}^{n \times h}$ we retain. Using the same pointwise multiplication trick as before, we arrive at the following update equation:

$$\mathbf{C}_t = \mathbf{F}_t \odot \mathbf{C}_{t-1} + \mathbf{I}_t \odot \tilde{\mathbf{C}}_t.$$

If the forget gate is always approximately 1 and the input gate is always approximately 0, the past memory cells $\mathbf{C}_{t-1}$ will be saved over time and passed to the current time step.
This design is introduced to alleviate the vanishing gradient problem and to better capture
long range dependencies within sequences.

We thus arrive at the flow diagram in :numref:`lstm_2`.

![Computing the memory cell in an LSTM model.](https://github.com/d2l-ai/d2l-tensorflow-colab-classic/blob/master/img/lstm-2.svg?raw=1)

:label:`lstm_2`


### Hidden State

Last, we need to define how to compute the hidden state $\mathbf{H}_t \in \mathbb{R}^{n \times h}$. This is where the output gate comes into play. In LSTM it is simply a gated version of the $\tanh$ of the memory cell.
This ensures that the values of $\mathbf{H}_t$ are always in the interval $(-1, 1)$.

$$\mathbf{H}_t = \mathbf{O}_t \odot \tanh(\mathbf{C}_t).$$


Whenever the output gate approximates 1 we effectively pass all memory information through to the predictor, whereas for the output gate close to 0 we retain all the information only within the memory cell and perform no further processing.



:numref:`lstm_3` has a graphical illustration of the data flow.

![Computing the hidden state in an LSTM model.](https://github.com/d2l-ai/d2l-tensorflow-colab-classic/blob/master/img/lstm-3.svg?raw=1)
:label:`lstm_3`



## Implementation from Scratch

Now let us implement an LSTM from scratch.
As same as the experiments in :numref:`sec_rnn_scratch`,
we first load the time machine dataset.


In [ ]:
import tensorflow as tf
from d2l import tensorflow as d2l

batch_size, num_steps = 32, 35
train_iter, vocab = d2l.load_data_time_machine(batch_size, num_steps)

### [**Initializing Model Parameters**]

Next we need to define and initialize the model parameters. As previously, the hyperparameter `num_hiddens` defines the number of hidden units. We initialize weights following a Gaussian distribution with 0.01 standard deviation, and we set the biases to 0.


In [ ]:
def get_lstm_params(vocab_size, num_hiddens):
    num_inputs = num_outputs = vocab_size

    def normal(shape):
        return tf.Variable(tf.random.normal(shape=shape, stddev=0.01,
                                            mean=0, dtype=tf.float32))
    def three():
        return (normal((num_inputs, num_hiddens)),
                normal((num_hiddens, num_hiddens)),
                tf.Variable(tf.zeros(num_hiddens), dtype=tf.float32))

    W_xi, W_hi, b_i = three()  # Input gate parameters
    W_xf, W_hf, b_f = three()  # Forget gate parameters
    W_xo, W_ho, b_o = three()  # Output gate parameters
    W_xc, W_hc, b_c = three()  # Candidate memory cell parameters
    # Output layer parameters
    W_hq = normal((num_hiddens, num_outputs))
    b_q = tf.Variable(tf.zeros(num_outputs), dtype=tf.float32)
    # Attach gradients
    params = [W_xi, W_hi, b_i, W_xf, W_hf, b_f, W_xo, W_ho, b_o, W_xc, W_hc,
              b_c, W_hq, b_q]
    return params

### Defining the Model

In [**the initialization function**], the hidden state of the LSTM needs to return an *additional* memory cell with a value of 0 and a shape of (batch size, number of hidden units). Hence we get the following state initialization.


In [ ]:
def init_lstm_state(batch_size, num_hiddens):
    return (tf.zeros(shape=(batch_size, num_hiddens)),
            tf.zeros(shape=(batch_size, num_hiddens)))

[**The actual model**] is defined just like what we discussed before: providing three gates and an auxiliary memory cell. Note that only the hidden state is passed to the output layer. The memory cell $\mathbf{C}_t$ does not directly participate in the output computation.


In [ ]:
def lstm(inputs, state, params):
    W_xi, W_hi, b_i, W_xf, W_hf, b_f, W_xo, W_ho, b_o, W_xc, W_hc, b_c, W_hq, b_q = params
    (H, C) = state
    outputs = []
    for X in inputs:
        X=tf.reshape(X,[-1,W_xi.shape[0]])
        I = tf.sigmoid(tf.matmul(X, W_xi) + tf.matmul(H, W_hi) + b_i)
        F = tf.sigmoid(tf.matmul(X, W_xf) + tf.matmul(H, W_hf) + b_f)
        O = tf.sigmoid(tf.matmul(X, W_xo) + tf.matmul(H, W_ho) + b_o)
        C_tilda = tf.tanh(tf.matmul(X, W_xc) + tf.matmul(H, W_hc) + b_c)
        C = F * C + I * C_tilda
        H = O * tf.tanh(C)
        Y = tf.matmul(H, W_hq) + b_q
        outputs.append(Y)
    return tf.concat(outputs, axis=0), (H,C)

### [**Training**] and Prediction

Let us train an LSTM as same as what we did in :numref:`sec_gru`, by instantiating the `RNNModelScratch` class as introduced in :numref:`sec_rnn_scratch`.


In [ ]:
vocab_size, num_hiddens, device_name = len(vocab), 256, d2l.try_gpu()._device_name
num_epochs, lr = 500, 1
strategy = tf.distribute.OneDeviceStrategy(device_name)
with strategy.scope():
    model = d2l.RNNModelScratch(len(vocab), num_hiddens, init_lstm_state, lstm, get_lstm_params)
d2l.train_ch8(model, train_iter, vocab, lr, num_epochs, strategy)

## [**Concise Implementation**]

Using high-level APIs,
we can directly instantiate an `LSTM` model.
This encapsulates all the configuration details that we made explicit above. The code is significantly faster as it uses compiled operators rather than Python for many details that we spelled out in detail before.


In [ ]:
lstm_cell = tf.keras.layers.LSTMCell(num_hiddens,
    kernel_initializer='glorot_uniform')
lstm_layer = tf.keras.layers.RNN(lstm_cell, time_major=True,
    return_sequences=True, return_state=True)
device_name = d2l.try_gpu()._device_name
strategy = tf.distribute.OneDeviceStrategy(device_name)
with strategy.scope():
    model = d2l.RNNModel(lstm_layer, vocab_size=len(vocab))
d2l.train_ch8(model, train_iter, vocab, lr, num_epochs, strategy)

LSTMs are the prototypical latent variable autoregressive model with nontrivial state control.
Many variants thereof have been proposed over the years, e.g., multiple layers, residual connections, different types of regularization. However, training LSTMs and other sequence models (such as GRUs) are quite costly due to the long range dependency of the sequence.
Later we will encounter alternative models such as transformers that can be used in some cases.


## Summary

* LSTMs have three types of gates: input gates, forget gates, and output gates that control the flow of information.
* The hidden layer output of LSTM includes the hidden state and the memory cell. Only the hidden state is passed into the output layer. The memory cell is entirely internal.
* LSTMs can alleviate vanishing and exploding gradients.


## Exercises

1. Adjust the hyperparameters and analyze the their influence on running time, perplexity, and the output sequence.
1. How would you need to change the model to generate proper words as opposed to sequences of characters?
1. Compare the computational cost for GRUs, LSTMs, and regular RNNs for a given hidden dimension. Pay special attention to the training and inference cost.
1. Since the candidate memory cell ensures that the value range is between $-1$ and $1$ by  using the $\tanh$ function, why does the hidden state need to use the $\tanh$ function again to ensure that the output value range is between $-1$ and $1$?
1. Implement an LSTM model for time series prediction rather than character sequence prediction.


## Exercise 1: Adjust the hyperparameters and analyze their influence on running time, perplexity, and the output sequence.

In [ ]:
# Adjust hyperparameters
num_hiddens = 128  # Number of hidden units
num_epochs = 300   # Number of epochs
lr = 0.5           # Learning rate

# Train the model with new hyperparameters
strategy = tf.distribute.OneDeviceStrategy(device_name)
with strategy.scope():
    model = d2l.RNNModelScratch(len(vocab), num_hiddens, init_lstm_state, lstm, get_lstm_params)
d2l.train_ch8(model, train_iter, vocab, lr, num_epochs, strategy)

## Exercise 2: Change the model to generate proper words as opposed to sequences of characters.

In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np

# Example text data
text = "This is an example text for the LSTM model. This is another sentence."

# Tokenize the text into words
tokenizer = Tokenizer()
tokenizer.fit_on_texts([text])
sequences = tokenizer.texts_to_sequences([text])
word_index = tokenizer.word_index

# Prepare the data
vocab_size = len(word_index) + 1
max_length = max(len(seq) for seq in sequences)
padded_sequences = pad_sequences(sequences, maxlen=max_length, padding='post')

# Split the data into input and target sequences
X = padded_sequences[:, :-1]
y = padded_sequences[:, 1:]

# Build the LSTM model using tf.keras
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(input_dim=vocab_size, output_dim=256, input_length=max_length-1),
    tf.keras.layers.LSTM(256, return_sequences=True),
    tf.keras.layers.Dense(vocab_size, activation='softmax')
])

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy')

# Train the model
model.fit(X, y, epochs=500, verbose=2)

# Function to generate text
def generate_text(model, tokenizer, seed_text, num_words):
    for _ in range(num_words):
        token_list = tokenizer.texts_to_sequences([seed_text])[0]
        token_list = pad_sequences([token_list], maxlen=max_length-1, padding='post')
        predicted = np.argmax(model.predict(token_list), axis=-1)
        output_word = tokenizer.index_word[int(predicted[0][-1])]
        seed_text += " " + output_word
    return seed_text

# Generate text
print(generate_text(model, tokenizer, "This is", 10))

## Exercise 3: Compare the computational cost for GRUs, LSTMs, and regular RNNs for a given hidden dimension.

In [ ]:
import tensorflow as tf
import time

# Define the hidden dimension and input size
hidden_dim = 256
input_dim = 256
batch_size = 32
time_steps = 35

# Generate random data for testing
inputs = tf.random.normal((batch_size, time_steps, input_dim))

# Function to measure time taken by a model
def measure_model_runtime(model, inputs, iterations=100):
    start_time = time.time()
    for _ in range(iterations):
        _ = model(inputs)
    end_time = time.time()
    avg_time = (end_time - start_time) / iterations
    return avg_time

# Define and measure runtime for a regular RNN model
rnn_model = tf.keras.Sequential([
    tf.keras.layers.SimpleRNN(hidden_dim, return_sequences=True)
])
rnn_time = measure_model_runtime(rnn_model, inputs)
print(f"Average computation time per iteration (RNN): {rnn_time:.5f} seconds")

# Define and measure runtime for a GRU model
gru_model = tf.keras.Sequential([
    tf.keras.layers.GRU(hidden_dim, return_sequences=True)
])
gru_time = measure_model_runtime(gru_model, inputs)
print(f"Average computation time per iteration (GRU): {gru_time:.5f} seconds")

# Define and measure runtime for an LSTM model
lstm_model = tf.keras.Sequential([
    tf.keras.layers.LSTM(hidden_dim, return_sequences=True)
])
lstm_time = measure_model_runtime(lstm_model, inputs)
print(f"Average computation time per iteration (LSTM): {lstm_time:.5f} seconds")

# Compare results
print("\nComparison of computational costs:")
print(f"RNN time: {rnn_time:.5f} seconds")
print(f"GRU time: {gru_time:.5f} seconds")
print(f"LSTM time: {lstm_time:.5f} seconds")


## Exercise 4: Why does the hidden state need to use the $\tanh$ function again to ensure that the output value range is between $-1$ and $1$?

In RNNs, GRUs, and LSTMs, the hidden state often passes through a `tanh` activation function to ensure that its values lie within the range \([-1, 1]\). This is done for several important reasons:

### 1. **Control of Hidden State Values**
   - The `tanh` function outputs values between \(-1\) and \(1\), which prevents the hidden states from growing too large. This is important because unbounded hidden states could lead to instability and potential issues with exploding gradients, especially in longer sequences.
   - By keeping the hidden state within a limited range, `tanh` helps maintain numerical stability and prevents the model from producing extremely high or low values that could disrupt learning.

### 2. **Non-linearity and Signal Modulation**
   - The `tanh` function introduces non-linearity into the model. This non-linearity allows the network to learn more complex patterns and dependencies in the data, as linear transformations alone cannot capture intricate relationships.
   - The bounded output of `tanh` also acts as a modulator for information flow. It allows the model to "forget" or "retain" certain aspects of previous information by controlling how much of the previous hidden state influences the current hidden state. This improves the model’s ability to handle long-range dependencies.

### 3. **Gradient Stability**
   - `tanh` helps mitigate the vanishing gradient problem. The bounded nature of the `tanh` function reduces the risk of large gradients, which could destabilize the training process. Moreover, by keeping the gradients in a moderate range, `tanh` facilitates more stable weight updates and promotes efficient learning.

### 4. **Consistency Across Layers**
   - In networks with multiple recurrent layers, using a consistent output range (such as \([-1, 1]\) with `tanh`) ensures that each layer’s hidden state is on a comparable scale. This consistency helps maintain signal clarity and promotes efficient learning across layers, improving the model's ability to learn meaningful patterns.

Thus, by using the `tanh` function to keep the hidden state within \([-1, 1]\), recurrent networks become more stable, learnable, and capable of handling complex sequential data.

## Exercise 5: Implement an LSTM model for time series prediction rather than character sequence prediction.

In [ ]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

# Generate synthetic time series data (e.g., a sine wave)
def generate_time_series(batch_size, n_steps):
    freq1, freq2, offset1, offset2 = np.random.rand(4, batch_size, 1)
    time = np.linspace(0, 1, n_steps)
    series = 0.5 * np.sin((time - offset1) * (freq1 * 10 + 10))   # Wave 1
    series += 0.2 * np.sin((time - offset2) * (freq2 * 20 + 20))  # Wave 2
    series += 0.1 * (np.random.rand(batch_size, n_steps) - 0.5)   # Noise
    return series[..., np.newaxis].astype(np.float32)

# Parameters
batch_size = 32
n_steps = 50

# Generate the data
series = generate_time_series(batch_size, n_steps + 1)
X_train, y_train = series[:, :-1], series[:, -1]

# Visualize a sample of the time series
plt.plot(X_train[0])
plt.xlabel("Time Steps")
plt.ylabel("Value")
plt.title("Sample Time Series")
plt.show()


In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense

# Define LSTM model for time series prediction
model = Sequential([
    LSTM(50, activation='relu', input_shape=(n_steps, 1)),
    Dense(1)  # Predict the value at the next time step
])

model.compile(optimizer='adam', loss='mse')
model.summary()

In [ ]:
# Train the model
history = model.fit(X_train, y_train, epochs=20, batch_size=32, validation_split=0.2)

In [ ]:
# Generate a new series for testing
X_test, y_test = generate_time_series(batch_size, n_steps + 1)[:, :-1], generate_time_series(batch_size, n_steps + 1)[:, -1]

# Predict the next value in the sequence
y_pred = model.predict(X_test)

# Visualize the predictions versus the actual values
plt.plot(y_test, label="Actual")
plt.plot(y_pred, label="Predicted")
plt.xlabel("Sample")
plt.ylabel("Value")
plt.legend()
plt.title("Time Series Prediction")
plt.show()